<a href="https://colab.research.google.com/github/tcae/FollowUpward/blob/master/cpc_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [6]:
%env PYTHONPATH=/env/python:FollowUpward
%env PYTHONPATH
#%env PYTHONPATH=$PYTHONPATH:FollowUpward
#%env PYTHONPATH


env: PYTHONPATH=/env/python:FollowUpward


'/env/python:FollowUpward'

In [11]:
!ls FollowUpward






Catalyst2PandasData.ipynb	logistic_regression.ipynb
catalyst_frame.py		__pycache__
ClassifyTimeSpecific.py		README.md
cpc_classifier.py		snippets.py
cpc_linear.ipynb		targets_features.py
currency_pairs_2_classifier.py	TestSnippets.ipynb
currency_pairs_mod.py		test_targets_features.py
df-test.pydata


In [3]:
!pwd


/content


In [5]:
!printenv PYTHONPATH


$PYTHONPATH:FollowUpward


In [8]:
!rm -R FollowUpward
!git clone https://github.com/tcae/FollowUpward.git

Cloning into 'FollowUpward'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 113 (delta 66), reused 61 (delta 29), pack-reused 0
Receiving objects: 100% (113/113), 13.75 MiB | 16.09 MiB/s, done.
Resolving deltas: 100% (66/66), done.


In [11]:
!git fetch FollowUpward.git


fatal: not a git repository (or any of the parent directories): .git


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
sys.path.append("./FollowUpward")


In [17]:
#from importlib.machinery import SourceFileLoader
#somemodule = SourceFileLoader('cpc_classifier', 'FollowUpward/cpc_classifier.py').load_module()
#somemodule = SourceFileLoader('TfVectors', 'FollowUpward/targets_features.py').load_module()
# from cpc_classifier import execute
%cd FollowUpward
!ls

import cpc_classifier
execute()

/content/FollowUpward
Catalyst2PandasData.ipynb	logistic_regression.ipynb
catalyst_frame.py		__pycache__
ClassifyTimeSpecific.py		README.md
cpc_classifier.py		snippets.py
cpc_linear.ipynb		targets_features.py
currency_pairs_2_classifier.py	TestSnippets.ipynb
currency_pairs_mod.py		test_targets_features.py
df-test.pydata


ImportError: ignored

In [18]:
!tail targets_features.py


                self.tf_vectors[time_agg][tgt + 'B'] = self.tf_aggs[time_agg].bottom.shift(offset)
                self.tf_vectors[time_agg][tgt + 'V'] = self.tf_aggs[time_agg].vol.shift(offset)
            self.tf_vectors[time_agg].dropna(inplace=True)


#if __name__ == "__main__":
#    import sys
#    fib(int(sys.argv[1]))
#    currency_data = FeaturesTargets()
#    print(currency_data.performances())
